In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import sklearn.metrics as metrics
import math


train_path="../input/house-prices-advanced-regression-techniques/train.csv"
test_path="../input/house-prices-advanced-regression-techniques/test.csv"

train=pd.read_csv(train_path)
test=pd.read_csv(test_path)

In [ ]:
#Copying Train And Test Datatset
c_train=train.copy()
c_test=test.copy()

In [ ]:
train.shape

In [ ]:
#To see the null values using a heatmap
#plt.figure(figsize = (16,7))
#sns.heatmap(train.isnull(),yticklabels=False,cbar=False)

* Lets Concat Train And Test Dataset

In [ ]:
c_train["train"]=1
c_test["train"]=0
df=pd.concat([c_train,c_test],axis=0,sort=False)
#Vertically Concat

In [ ]:
df.shape

83 = train/test features+train id +test id+SalePrice

In [ ]:
df

* We Have Added Two New Features Train and Test To Show Which Values Are Present In Either Train Or Test

* Calculating the percentage of missing values of each feature

In [ ]:
NAN=[(k,df[k].isnull().mean()*100) for k in df]
print(NAN)

In [ ]:
#Making a new dataframe
NAN=pd.DataFrame(NAN,columns=["column_name","percentage"])

* Displaying Features Which Has More Than 50% Null Values

In [ ]:
NAN=NAN[NAN.percentage>50]
NAN.sort_values("percentage",ascending=False)

In [ ]:
#Dropping All The Above Features
df=df.drop(["PoolQC","MiscFeature","Alley","Fence"],axis=1)

* Now we will select numerical and categorical features 

In [ ]:
cat_columns = df.select_dtypes(include=['O'])
num_columns =df.select_dtypes(exclude=['O'])

# Dealing With Categorical Columns

In [ ]:
null_cat_columns=cat_columns.isnull().sum()
null_cat_columns

We will fill -- **BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, GarageType, GarageFinish, GarageQual, FireplaceQu, GarageCond** -- with "None" (Check The Above Output,We Place The Node In Features Where There Are Many NULL Values).

We will fill the rest of features with th most frequent value (using its own most frequent value).

In [ ]:
columns_None = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
                'GarageType','GarageFinish','GarageQual','FireplaceQu','GarageCond']
cat_columns[columns_None]= cat_columns[columns_None].fillna('None')

In [ ]:
mode_cols = ['MSZoning','Utilities','Exterior1st','Exterior2nd',
             'MasVnrType','Electrical','KitchenQual','Functional','SaleType']

#fill missing values for each column (using its own most frequent value)
cat_columns[mode_cols] = cat_columns[mode_cols].fillna(cat_columns.mode().iloc[0])

In [ ]:
null_cat_columns=cat_columns.isnull().sum()
null_cat_columns

# Dealing With Num Vals

In [ ]:
null_num_columns=num_columns.isnull().sum()
null_num_columns

**By The Above Table We Can See That We Gotta Fill LotFrontage and GarageYrBlt**

In [ ]:
print((num_columns["GarageYrBlt"]).median())
print(num_columns["LotFrontage"].median())

*We Can Use Either Mean Or Median Depending Upon Us*

In [ ]:
num_columns['GarageYrBlt'] = num_columns['GarageYrBlt'].fillna(1979)
num_columns['LotFrontage'] = num_columns['LotFrontage'].fillna(68)

Lets Fill The Other Null Num Features With 0

In [ ]:
num_columns=num_columns.fillna(0)

In [ ]:
num_columns.isnull().sum()

# Lets Make Some New Features

In [ ]:
num_columns["House_Age"]= num_columns['YrSold']- num_columns['YearBuilt']
num_columns["House_Age"].describe()

* Lets Find If There Is A Negative Value In House_Age

In [ ]:
neg_house=num_columns[num_columns["House_Age"]<0]
neg_house

In [ ]:
neg_house['YrSold']

* Like we see here that the minimun is -1 ???
* It is strange to find that the house was sold in 2007 and built in 2008.
* So we decide to change the year of sold to 2009

In [ ]:
num_columns.loc[num_columns["YrSold"]<num_columns["YearRemodAdd"],"YrSold"]=2009

num_columns["House_Age"]= num_columns['YrSold']- num_columns['YearBuilt']
num_columns["House_Age"].describe()

**According To The Feature Description**

* TotalBsmtBath : Sum of : 1/2 * BsmtFullBath + BsmtHalfBath

* TotalBath : Sum of : 1/2 * FullBath and HalfBath

* TotalSA : Sum of : 1stFlrSF and 2ndFlrSF and TotalBsmtSF

In [ ]:
num_columns['TotalBsmtBath'] = num_columns['BsmtFullBath']*0.5 + num_columns['BsmtHalfBath']
num_columns['TotalBath'] = num_columns['FullBath']*0.5  + num_columns['HalfBath']
num_columns['TotalSA']=num_columns['TotalBsmtSF'] + num_columns['1stFlrSF'] + num_columns['2ndFlrSF']

In [ ]:
num_columns.head()

# Lets Encode Categorical Values

In [ ]:
cat_columns.head()

In [ ]:
#Using One Hot Encoder
#cat_columns=pd.get_dummies(cat_columns,columns=cat_columns.columns)

* It simply creates additional features based on the number of unique values in the categorical feature. Every unique value in the category will be added as a feature.*

In [ ]:
#Using LabelEncoder
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

for features in cat_columns.columns:
    cat_columns[features]=encoder.fit_transform(cat_columns[features])

*Label Encoding is a popular encoding technique for handling categorical variables. In this technique, each label is assigned a unique integer based on alphabetical ordering.*

In [ ]:
cat_columns.head()

# Now Lets Concat The Num And Cat Features

In [ ]:
df_final = pd.concat([cat_columns, num_columns], axis=1,sort=False)
df_final.head()

# Initialising Train,Test Data

In [ ]:
df_final = df_final.drop(['Id'],axis=1)

#We Had Initialised Train Data As 1 And Test Data As 0 In the Beggining
df_train = df_final[df_final['train'] == 1]
df_train = df_train.drop(['train'],axis=1)

#Vertically Splitting The Train And Test

df_test = df_final[df_final['train'] == 0]
df_test = df_test.drop(['SalePrice'],axis=1)
df_test = df_test.drop(['train'],axis=1)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

*Now We Separate Train And Target Features*

In [ ]:
target= df_train['SalePrice']
df_train = df_train.drop(['SalePrice'],axis=1)

# Modelling

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df_train,target,test_size=0.33,random_state=0)

**XGB Documentation**: https://xgboost.readthedocs.io/en/latest/parameter.html

**LGBM Documentation**: https://lightgbm.readthedocs.io/en/latest/Parameters.html#core-parameters

In [ ]:
xgb =XGBRegressor( booster='gbtree', colsample_bylevel=1,
                  colsample_bynode=1, colsample_bytree=0.6, gamma=0,
                  importance_type='gain', learning_rate=0.01, max_delta_step=0,
                  max_depth=4, min_child_weight=1.5, n_estimators=2400,
                  n_jobs=1, nthread=None, objective='reg:linear',
                  reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
                  silent=None, subsample=0.8, verbosity=1)


lgbm = LGBMRegressor(objective='regression', 
                     num_leaves=4,
                     learning_rate=0.01, 
                     n_estimators=12000, 
                     max_bin=200, 
                     bagging_fraction=0.75,
                     bagging_freq=5, 
                     bagging_seed=7,
                     feature_fraction=0.4)

I Found This Article Very Helpful To Know About LGBM And XGBoost
https://www.analyticsvidhya.com/blog/2017/06/which-algorithm-takes-the-crown-light-gbm-vs-xgboost/                                                

In [ ]:
#Fitting
xgb.fit(x_train, y_train)
lgbm.fit(x_train, y_train,eval_metric='rmse')

In [ ]:
predict1 = xgb.predict(x_test)
predict = lgbm.predict(x_test)

In [ ]:
print('Root Mean Square Error test(XGBOOST) = ' + str(math.sqrt(metrics.mean_squared_error(y_test, predict1))))
print('Root Mean Square Error test(LGBM) = ' + str(math.sqrt(metrics.mean_squared_error(y_test, predict))))

* Fitting With all the dataset 

In [ ]:
xgb.fit(df_train, target)
lgbm.fit(df_train, target,eval_metric='rmse')

Root Mean Square Error (RMSE) is a standard way to measure the error of a model in predicting quantitative data.

Merging Both Predictions (45% of XGB and 55% of LGBM)

In [ ]:
predict4 = lgbm.predict(df_test)
predict3 = xgb.predict(df_test)
predict_y = (predict3*0.45 + predict4*0.55)
print(predict_y)

In [ ]:
submission = pd.DataFrame({
        "Id": test["Id"],
        "SalePrice": predict_y
    })
submission.to_csv('submission.csv', index=False)